# Multivariate time series forecasting using LSTM using cycle-feature.

In [94]:
# %reset -f
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns
sns.set()
rcParams['figure.figsize'] = 10,10
from sklearn.preprocessing import StandardScaler 
from functionsAll import lstmMultiSplit, originalToRecurring, recurringToOrignal
import warnings
warnings.filterwarnings('ignore')

In [2]:
# cyDf['day'] = cyDf['day'].apply(lambda row: recurringToOrignal(rcUnits=row,period=32))


## Data cleaning.

In [19]:
df = pd.read_csv("../Data/cleanned/zusammen.csv", parse_dates=["date"])
df['dayofyear'] = df['date'].dt.dayofyear
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['weekday'] = df['date'].dt.weekday
df['hour'] = df['date'].dt.hour
df['day'] = df['date'].dt.day
df['week'] = df['date'].dt.week
df.drop(columns=['date','rohwert'], inplace= True)
# Conclude with 11 features, 1 label.
df = df[[ 'wert', 'day', 'month', 'dayofyear', 'year', 'weekday', 'week', 'hour','temp', 'dwpt', 'rhum', 'prcp']]
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

/home/cluster_home/yash/workspace/AutoMLRepo/Auto-ML-On-De-Pollution/autoMLVenv/lib64/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [20]:
df.head(10)

,wert,day,month,dayofyear,year,weekday,week,hour,temp,dwpt,rhum,prcp
0,36,2,1,2,2020,3,1,0,1.0,-0.6,89.0,0.0
1,31,2,1,2,2020,3,1,7,0.1,-1.2,91.0,0.0
2,32,2,1,2,2020,3,1,6,-0.3,-1.3,93.0,0.0
3,27,2,1,2,2020,3,1,5,-0.5,-1.2,95.0,0.0
4,38,2,1,2,2020,3,1,4,-0.4,-0.8,97.0,0.0
5,49,2,1,2,2020,3,1,3,0.4,-0.2,96.0,0.0
6,46,2,1,2,2020,3,1,2,0.9,-0.3,92.0,0.0
7,38,2,1,2,2020,3,1,1,0.9,-0.6,90.0,0.0
8,33,2,1,2,2020,3,1,8,0.1,-1.3,90.0,0.0
9,34,2,1,2,2020,3,1,9,0.2,-1.4,89.0,0.0


In [21]:
cyDf = df[['day','month','dayofyear','weekday', 'week', 'hour']] # [32,13,367,7,54,52]
scDf = df[['wert','year' , 'temp', 'dwpt', 'rhum', 'prcp']]

### Testing of functions

In [32]:
day = cyDf.day.unique()
day.sort()
rcDay = originalToRecurring(ogUnits=day,period=32)
ogDay = recurringToOrignal(rcUnits=rcDay,period=32)
day == ogDay

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [36]:
month = cyDf.month.unique()
month.sort()
rcmonth = originalToRecurring(ogUnits=month,period=13)
ogmonth = recurringToOrignal(rcUnits=rcmonth,period=13)
month == ogmonth

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [39]:
dayofyear = cyDf.dayofyear.unique()
dayofyear.sort()
rcdayofyear = originalToRecurring(ogUnits=dayofyear,period=367)
ogdayofyear = recurringToOrignal(rcUnits=rcdayofyear,period=367)
dayofyear == ogdayofyear

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [91]:
weekday = cyDf.weekday.unique()
weekday.sort()
rcweekday = originalToRecurring(ogUnits=weekday,period=7)
ogweekday = recurringToOrignal(rcUnits=rcweekday,period=7)
weekday

array([0, 1, 2, 3, 4, 5, 6])

In [49]:
week = cyDf.week.unique()
week.sort()
rcweek = originalToRecurring(ogUnits=week,period=54)
ogweek = recurringToOrignal(rcUnits=rcweek,period=54)
week == ogweek

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [55]:
hour = cyDf.hour.unique()
hour.sort()
rchour = originalToRecurring(ogUnits=hour,period=25)
oghour = recurringToOrignal(rcUnits=rchour,period=25)
hour == oghour

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [142]:
rchour[1]

array([0.24868989, 0.96858316])

## Data normalizing

In [95]:
cyDf['day']=cyDf['day'].apply(lambda row: originalToRecurring(ogUnits=row,period=32))
cyDf['month']=cyDf['month'].apply(lambda row: originalToRecurring(ogUnits=row,period=13))
cyDf['dayofyear']=cyDf['dayofyear'].apply(lambda row: originalToRecurring(ogUnits=row,period=367))
cyDf['weekday']=cyDf['weekday'].apply(lambda row: originalToRecurring(ogUnits=row,period=7))
cyDf['week']=cyDf['week'].apply(lambda row: originalToRecurring(ogUnits=row,period=54))
cyDf['hour']=cyDf['hour'].apply(lambda row: originalToRecurring(ogUnits=row,period=25))

In [143]:
cyDf.hour[1].shape

(1, 4)

In [102]:
scaler = StandardScaler()
scaler = scaler.fit(scDf)
scaledDf = scaler.transform(scDf)

### Combine in one dataframe. [cycle + normalize]

In [105]:
scaledDf.shape, cyDf.shape

((23736, 6), (23736, 6))

In [108]:
scaledDf = pd.DataFrame({'wert':scaledDf[0],
                        'year':scaledDf[1],
                        'temp':scaledDf[2],
                        'dwpt':scaledDf[3],
                        'rhum':scaledDf[4],
                        'prcp':scaledDf[5]})

In [112]:
mainDf = pd.concat([scaledDf, cyDf], axis=1)
mainDf.head()

,wert,year,temp,dwpt,rhum,prcp,day,month,dayofyear,weekday,week,hour
0,1.680347,1.188967,1.287243,0.795863,1.876899,2.957935,"[[0.014739235850982756, 0.03541602186592587, 0...","[[0.22272706085110822, 0.4150159464378762, 0.9...","[[0.0005861013372221497, 0.01710952579237599, ...","[[0.3796825565373579, -0.7233955756832762, 0.9...","[[0.013507613141101035, 0.11531144521560933, 0...","[[0.0, 0.2486898871648548, 1.0, 0.968583161128..."
1,-1.159369,-1.159369,-1.159369,-1.159369,-1.159369,-1.159369,"[[0.014739235850982756, 0.03541602186592587, 0...","[[0.22272706085110822, 0.4150159464378762, 0.9...","[[0.0005861013372221497, 0.01710952579237599, ...","[[0.3796825565373579, -0.7233955756832762, 0.9...","[[0.013507613141101035, 0.11531144521560933, 0...","[[0.24437559607395543, -0.04707665490445079, 0..."
2,-1.566281,-1.691633,-1.747346,-1.775202,-1.761274,-1.649849,"[[0.014739235850982756, 0.03541602186592587, 0...","[[0.22272706085110822, 0.4150159464378762, 0.9...","[[0.0005861013372221497, 0.01710952579237599, ...","[[0.3796825565373579, -0.7233955756832762, 0.9...","[[0.013507613141101035, 0.11531144521560933, 0...","[[0.24820950014387178, 0.015780323783663428, 0..."
3,-1.385630,-1.490882,-1.508424,-1.490882,-1.420714,-1.315462,"[[0.014739235850982756, 0.03541602186592587, 0...","[[0.22272706085110822, 0.4150159464378762, 0.9...","[[0.0005861013372221497, 0.01710952579237599, ...","[[0.3796825565373579, -0.7233955756832762, 0.9...","[[0.013507613141101035, 0.11531144521560933, 0...","[[0.23675698752108199, 0.07758638947605337, 0...."
4,0.792786,0.903869,1.014952,1.126034,1.237117,1.181576,"[[0.014739235850982756, 0.03541602186592587, 0...","[[0.22272706085110822, 0.4150159464378762, 0.9...","[[0.0005861013372221497, 0.01710952579237599, ...","[[0.3796825565373579, -0.7233955756832762, 0.9...","[[0.013507613141101035, 0.11531144521560933, 0...","[[0.21061375254563916, 0.13426128651581468, 0...."


## Data windowing and splitting

In [125]:
mianNp = mainDf.to_numpy()

In [120]:
mainDf.shape

(23736, 12)

In [123]:
mainDf[1:0]

,wert,year,temp,dwpt,rhum,prcp,day,month,dayofyear,weekday,week,hour


In [126]:
train = mianNp[:23000]
test = mianNp[23000:] 

In [127]:
trainX, trainY = lstmMultiSplit(df=train, n_past=5, n_future=1)
testX, testY = lstmMultiSplit(df=test, n_past=5, n_future=1)

In [129]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

history = model.fit(trainX, trainY, epochs=30, batch_size=14, validation_split=0.1, verbose=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 5, 128)            72192     
_________________________________________________________________
lstm_4 (LSTM)                (None, 5, 64)             49408     
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 134,049
Trainable params: 134,049
Non-trainable params: 0
_________________________________________________________________


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).